In [1791]:
import pandas as pd
import numpy as np
import seaborn as sb
import pylab as py
import matplotlib.pyplot as plt
import sklearn.model_selection as mods
import sklearn.linear_model as sklin
import sklearn.tree as st

In [1792]:
df=pd.read_csv('./TestPredictionResults_23-03-14_01-28-53.csv')

In [1793]:
df.head()

,TestProgram_Name_NA,Family,IsConcurrent,Patterns_Count,Tests_Count,Mtt_Count,ConcurrentFlows_Count,Shmoo_tests_count,ITuff_Temperature_NA,ITuff_SubmitterFullName_NA,ITuff_Lot_NA,ituff_EndDate_NA,ITuff_PartType_FromSpark,ITuff_BomGroup_FromSpark,ITuff_ProcessStep_FromSpark,ITuff_ExperimentType_FromSpark,ITuff_PerUnit_IsPassed_Target_NA,ITuff_PerUnit_testTimeInMS_Target
0,RPL_1ST_SILICON,RPL,False,20344,14787,0,0,248,0,"Savadatti, Shravanakumar",Y311024CR,14/03/2023 12:49:55,H64BJQFVAB,CLASS_RPL8161S_BGA,PHMHOT,Correlation,True,212.687689
1,RPL_1ST_SILICON,RPL,False,20344,14787,0,0,248,0,"Savadatti, Shravanakumar",Y311024CR,14/03/2023 12:49:55,H64BJQFVAB,CLASS_RPL8161S_BGA,PHMHOT,Correlation,True,212.619955
2,RPL_1ST_SILICON,RPL,False,20344,14787,0,0,248,0,"Savadatti, Shravanakumar",Y311024CR,14/03/2023 12:49:55,H64BJQFVAB,CLASS_RPL8161S_BGA,PHMHOT,Correlation,True,212.731636
3,RPL_1ST_SILICON,RPL,False,20344,14787,0,0,248,0,"Savadatti, Shravanakumar",Y311024CR,14/03/2023 12:49:55,H64BJQFVAB,CLASS_RPL8161S_BGA,PHMHOT,Correlation,True,212.529103
4,RPL_1ST_SILICON,RPL,False,20344,14787,0,0,248,0,"Savadatti, Shravanakumar",Y311024CR,14/03/2023 12:49:55,H64BJQFVAB,CLASS_RPL8161S_BGA,PHMHOT,Correlation,True,209.062365


In [1794]:
#df = df.drop(df[df['ITuff_PerUnit_IsPassed_Target_NA'] == False].index)


In [1795]:
#sb.pairplot(df) 

In [1796]:
#sb.displot(df['Shmoo_tests_count'])

In [1797]:
#df.corr()

In [1798]:
df.describe()

,Patterns_Count,Tests_Count,Mtt_Count,ConcurrentFlows_Count,Shmoo_tests_count,ITuff_Temperature_NA,ITuff_PerUnit_testTimeInMS_Target
count,6458.000000,6458.000000,6458.0,6458.000000,6458.000000,6458.000000,6458.000000
mean,17521.001703,13149.794828,0.0,1.214772,243.851502,64.060081,264.893140
std,5689.697092,4205.389891,0.0,1.472746,114.296177,48.280555,114.235861
min,5704.000000,1847.000000,0.0,0.000000,22.000000,-5.000000,12.365689
25%,16006.000000,13436.000000,0.0,0.000000,239.000000,0.000000,194.753946
50%,17458.000000,14787.000000,0.0,0.000000,248.000000,95.000000,229.296788
75%,20344.000000,15550.000000,0.0,3.000000,313.000000,100.000000,351.642295
max,38520.000000,19079.000000,0.0,3.000000,577.000000,100.000000,1094.148635


In [1799]:
df.groupby('IsConcurrent').count()

,TestProgram_Name_NA,Family,Patterns_Count,Tests_Count,Mtt_Count,ConcurrentFlows_Count,Shmoo_tests_count,ITuff_Temperature_NA,ITuff_SubmitterFullName_NA,ITuff_Lot_NA,ituff_EndDate_NA,ITuff_PartType_FromSpark,ITuff_BomGroup_FromSpark,ITuff_ProcessStep_FromSpark,ITuff_ExperimentType_FromSpark,ITuff_PerUnit_IsPassed_Target_NA,ITuff_PerUnit_testTimeInMS_Target
IsConcurrent,,,,,,,,,,,,,,,,,
False,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843,3843
True,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615,2615


In [1800]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6458 entries, 0 to 7037
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   TestProgram_Name_NA                6458 non-null   object 
 1   Family                             6458 non-null   object 
 2   IsConcurrent                       6458 non-null   bool   
 3   Patterns_Count                     6458 non-null   int64  
 4   Tests_Count                        6458 non-null   int64  
 5   Mtt_Count                          6458 non-null   int64  
 6   ConcurrentFlows_Count              6458 non-null   int64  
 7   Shmoo_tests_count                  6458 non-null   int64  
 8   ITuff_Temperature_NA               6458 non-null   int64  
 9   ITuff_SubmitterFullName_NA         6458 non-null   object 
 10  ITuff_Lot_NA                       6458 non-null   object 
 11  ituff_EndDate_NA                   6458 non-null   objec

In [1801]:
df.drop(['TestProgram_Name_NA','ITuff_Temperature_NA','ITuff_SubmitterFullName_NA','ITuff_Lot_NA', 'ituff_EndDate_NA','ITuff_PerUnit_IsPassed_Target_NA'], axis=1, inplace=True)
df.drop(['ConcurrentFlows_Count'], axis=1, inplace=True)                # ConcurrentFlows_Count doesn't change result
df.drop(['IsConcurrent'], axis=1, inplace=True)                         # 'IsConcurrent' doesn’t change result
df.drop(['Family'], axis=1, inplace=True)                               # 'Family' doesn’t change result
#df.drop(['Patterns_Count'], axis=1, inplace=True)                       # Patterns_Count helps a little
#df.drop(['Tests_Count'], axis=1, inplace=True)                         # Tests_Count helps a little
#df.drop(['Shmoo_tests_count'], axis=1, inplace=True)                   # Shmoo_tests_count helps a little
df.drop(['ITuff_BomGroup_FromSpark'], axis=1, inplace=True)
#df.drop(['ITuff_ProcessStep_FromSpark'], axis=1, inplace=True)         # ITuff_ProcessStep_FromSpark is critical
#df.drop(['ITuff_ExperimentType_FromSpark'], axis=1, inplace=True)      # ITuff_ExperimentType_FromSpark helps a little

In [1802]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6458 entries, 0 to 7037
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Patterns_Count                     6458 non-null   int64  
 1   Tests_Count                        6458 non-null   int64  
 2   Mtt_Count                          6458 non-null   int64  
 3   Shmoo_tests_count                  6458 non-null   int64  
 4   ITuff_PartType_FromSpark           6458 non-null   object 
 5   ITuff_ProcessStep_FromSpark        6458 non-null   object 
 6   ITuff_ExperimentType_FromSpark     6458 non-null   object 
 7   ITuff_PerUnit_testTimeInMS_Target  6458 non-null   float64
dtypes: float64(1), int64(4), object(3)
memory usage: 454.1+ KB


In [1803]:
#df = pd.get_dummies(df, columns =['Family'])
df = pd.get_dummies(df, columns =['ITuff_PartType_FromSpark'])
#df = pd.get_dummies(df, columns =['ITuff_BomGroup_FromSpark'])
df = pd.get_dummies(df, columns =['ITuff_ProcessStep_FromSpark'])
df = pd.get_dummies(df, columns =['ITuff_ExperimentType_FromSpark'])


In [1804]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6458 entries, 0 to 7037
Data columns (total 40 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Patterns_Count                              6458 non-null   int64  
 1   Tests_Count                                 6458 non-null   int64  
 2   Mtt_Count                                   6458 non-null   int64  
 3   Shmoo_tests_count                           6458 non-null   int64  
 4   ITuff_PerUnit_testTimeInMS_Target           6458 non-null   float64
 5   ITuff_PartType_FromSpark_H64AHSFVCC         6458 non-null   uint8  
 6   ITuff_PartType_FromSpark_H64AQQFVAB         6458 non-null   uint8  
 7   ITuff_PartType_FromSpark_H64ASLBVQ          6458 non-null   uint8  
 8   ITuff_PartType_FromSpark_H64ASLKVQ          6458 non-null   uint8  
 9   ITuff_PartType_FromSpark_H64BJQAVAB         6458 non-null   uint8  
 10  ITuff_PartTy

In [1805]:
df.describe()

,Patterns_Count,Tests_Count,Mtt_Count,Shmoo_tests_count,ITuff_PerUnit_testTimeInMS_Target,ITuff_PartType_FromSpark_H64AHSFVCC,ITuff_PartType_FromSpark_H64AQQFVAB,ITuff_PartType_FromSpark_H64ASLBVQ,ITuff_PartType_FromSpark_H64ASLKVQ,ITuff_PartType_FromSpark_H64BJQAVAB,...,ITuff_PartType_FromSpark_P58ADTAVAB,ITuff_ProcessStep_FromSpark_CLASSCOLD,ITuff_ProcessStep_FromSpark_CLASSHOT,ITuff_ProcessStep_FromSpark_CSM,ITuff_ProcessStep_FromSpark_FUSE,ITuff_ProcessStep_FromSpark_PHMCOLD,ITuff_ProcessStep_FromSpark_PHMHOT,ITuff_ExperimentType_FromSpark_Correlation,ITuff_ExperimentType_FromSpark_Engineering,ITuff_ExperimentType_FromSpark_WalkTheLot
count,6458.000000,6458.000000,6458.0,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,...,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000
mean,17521.001703,13149.794828,0.0,243.851502,264.893140,0.052183,0.014556,0.044286,0.011459,0.082378,...,0.027253,0.213069,0.498606,0.004645,0.001548,0.108702,0.173428,0.731806,0.255342,0.012852
std,5689.697092,4205.389891,0.0,114.296177,114.235861,0.222414,0.119775,0.205746,0.106438,0.274962,...,0.162832,0.409508,0.500037,0.068004,0.039323,0.311290,0.378646,0.443054,0.436087,0.112646
min,5704.000000,1847.000000,0.0,22.000000,12.365689,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16006.000000,13436.000000,0.0,239.000000,194.753946,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17458.000000,14787.000000,0.0,248.000000,229.296788,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,20344.000000,15550.000000,0.0,313.000000,351.642295,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,38520.000000,19079.000000,0.0,577.000000,1094.148635,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [1806]:
#df.Family_RaptorLake.value_counts()

In [1807]:
#sb.pairplot(df,hue='ITuff_PerUnit_testTimeInMS_Target',)

In [1808]:
#plt.hist(df['ITuff_PerUnit_testTimeInMS_Target'])
#plt.show()

In [1809]:
#sb.displot(df['ITuff_PerUnit_testTimeInMS_Target']) 

In [1810]:
x = df.drop(['ITuff_PerUnit_testTimeInMS_Target'],axis=1)
y = df.ITuff_PerUnit_testTimeInMS_Target

In [1811]:
x_train, x_test, y_train, y_test = mods.train_test_split(x, y, test_size=0.30,random_state=101)

In [1812]:
model = sklin.LinearRegression()

In [1813]:
model.fit(x_train,y_train)

LinearRegression()

In [1814]:
y_pred = model.predict(x_test)

In [1815]:
import matplotlib.pyplot as plt

In [1816]:
#plt.scatter(y_test,y_pred)

In [1817]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [1818]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# print the results
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-squared: ", r2)
print("MAE: ", mae)

MSE:  1623157.037487776
RMSE:  1274.0318039545857
R-squared:  -124.88849138488843
MAE:  89.9904656925864
